In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import numpy as np
import json
import time

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util
import faiss

load_dotenv(verbose=True)

API_KEY = os.getenv('API_KEY')
client = OpenAI(api_key=API_KEY)

/Users/notaehyeong/miniforge3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Basic RAG

## Prepare documents and Embedding

In [3]:
# 1. 문서 데이터베이스 준비
documents = [
    "RAG combines retrieval and generation to enhance text generation tasks.",
    "It uses a retriever to fetch relevant documents for a given query.",
    "The generator then produces the final output based on the retrieved documents.",
    "This approach is especially useful for knowledge-intensive tasks."
]

# 2. Sentence-BERT로 문서 임베딩
retriever = SentenceTransformer('all-MiniLM-L6-v2') # change to openai embedding model
document_embeddings = retriever.encode(documents, convert_to_tensor=True)
document_embeddings

tensor([[-0.0490,  0.0476,  0.0423,  ...,  0.0570,  0.0271, -0.0106],
        [-0.0521,  0.0510, -0.0635,  ...,  0.0877,  0.0559,  0.0605],
        [-0.0994,  0.1269, -0.1022,  ...,  0.0842,  0.1130,  0.0064],
        [-0.0208,  0.0361, -0.0472,  ...,  0.0449, -0.0228,  0.0603]],
       device='mps:0')

## Make Faiss index

In [4]:
# 3. FAISS로 검색 인덱스 생성
dimension = document_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(document_embeddings.cpu()))
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x105863e40> >

## Query embedding

In [5]:
# 4. 질문 정의
query = "What is RAG and how does it work?"
query_embedding = retriever.encode(query, convert_to_tensor=True).cpu().numpy()
query_embedding

array([-8.71513635e-02,  7.33634606e-02,  3.93800028e-02,  1.04654785e-02,
       -6.15194142e-02,  5.47023350e-03,  5.94053306e-02,  6.10240139e-02,
       -1.78641174e-02, -3.20010968e-02, -9.42157023e-03,  4.57748771e-02,
        2.68447082e-02, -4.86106351e-02, -7.31136352e-02,  8.20814893e-02,
        6.39288574e-02,  1.02431938e-01, -1.98324285e-02, -3.61080207e-02,
       -1.20374588e-02,  3.08231898e-02, -2.75140628e-02, -4.36160602e-02,
        1.96681432e-02,  5.38504906e-02, -4.66142744e-02,  5.46863908e-03,
        6.40840530e-02, -1.74713172e-02,  9.10303928e-03,  4.85722125e-02,
       -3.18480544e-02, -2.24165376e-02, -6.25128746e-02,  1.96867865e-02,
       -2.03983709e-02,  5.88036850e-02, -1.53918294e-02,  3.69462259e-02,
        3.92769389e-02, -3.10819726e-02, -2.80910823e-02, -7.99766928e-02,
        2.43709255e-02,  3.64410095e-02, -2.06157775e-03,  4.76543419e-02,
        1.13168210e-02, -2.39392892e-02, -8.32374115e-03,  4.24382696e-03,
        4.91901115e-02,  

## Retrieval documents

In [7]:
# 5. 가장 유사한 문서 검색
k = 2  # Top-k 검색
distances, indices = index.search(query_embedding.reshape(1, -1), k)
distances

array([[1.0134497, 1.6782237]], dtype=float32)

## See results

In [25]:
# 검색된 문서 가져오기
retrieved_docs = [documents[i] for i in indices[0]]
retrieved_docs

['RAG combines retrieval and generation to enhance text generation tasks.',
 'It uses a retriever to fetch relevant documents for a given query.']

# Webpage RAG

In [11]:
from langchain_community.document_loaders import WebBaseLoader # Get webpage

url = 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8'
loader = WebBaseLoader(url)

docs = loader.load()# text-> Documents

print(len(docs))
print(len(docs[0].page_content))
print(docs[0].page_content[5000:6000])

1
13141
 위해 좀 더 빠르게 강력한 수단을 이용해야 합니다. 특히 정책 문서에 명시된 원칙을 지키지 않는 것은 대부분의 경우 다른 사용자에게 받아들여지지 않습니다 (다른 분들에게 예외 상황임을 설득할 수 있다면 가능하기는 하지만요). 이는 당신을 포함해서 편집자 개개인이 정책과 지침을 직접 집행 및 적용한다는 것을 의미합니다.
특정 사용자가 명백히 정책에 반하는 행동을 하거나 정책과 상충되는 방식으로 지침을 어기는 경우, 특히 의도적이고 지속적으로 그런 행위를 하는 경우 해당 사용자는 관리자의 제재 조치로 일시적, 혹은 영구적으로 편집이 차단될 수 있습니다. 영어판을 비롯한 타 언어판에서는 일반적인 분쟁 해결 절차로 끝낼 수 없는 사안은 중재위원회가 개입하기도 합니다.

문서 내용
정책과 지침의 문서 내용은 처음 읽는 사용자라도 원칙과 규범을 잘 이해할 수 있도록 다음 원칙을 지켜야 합니다.

명확하게 작성하세요. 소수만 알아듣거나 준법률적인 단어, 혹은 지나치게 단순한 표현은 피해야 합니다. 명확하고, 직접적이고, 모호하지 않고, 구체적으로 작성하세요. 지나치게 상투적인 표현이나 일반론은 피하세요. 지침, 도움말 문서 및 기타 정보문 문서에서도 "해야 합니다" 혹은 "하지 말아야 합니다" 같이 직접적인 표현을 굳이 꺼릴 필요는 없습니다.
가능한 간결하게, 너무 단순하지는 않게. 정책이 중언부언하면 오해를 부릅니다. 불필요한 말은 생략하세요. 직접적이고 간결한 설명이 마구잡이식 예시 나열보다 더 이해하기 쉽습니다. 각주나 관련 문서 링크를 이용하여 더 상세히 설명할 수도 있습니다.
규칙을 만든 의도를 강조하세요. 사용자들이 상식대로 행동하리라 기대하세요. 정책의 의도가 명료하다면, 추가 설명은 필요 없죠. 즉 규칙을 '어떻게' 지키는지와 더불어 '왜' 지켜야 하는지 확실하게 밝혀야 합니다.
범위는 분명히, 중복은 피하기. 되도록 앞부분에서 정책 및 지침의 목적과 범위를 분명하게 밝혀야 합니다. 독자 대부분은 도입부 초반만 읽고 나가버리니까요. 각 정책 

## Split documents to chunks

In [16]:
# Text Split (Documents -> small chunks: Documents)
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

print(len(splits))
print(splits[9])


18
page_content='채택 과정\n한국어 위키백과에서 오랫동안 확립되어 온 정책과 지침의 대다수는, 영어 위키백과 설립 시 토대가 된 원칙에서 발전된 것들입니다. 물론 타 언어 위키백과의 원칙을 가져오는 것 말고도, 정책과 지침을 일반적인 문제와 문서 훼손 행위의 대응책으로 한국어 위키백과 내 공동체에서 자발적으로 세우기도 했습니다. 정책과 지침 대부분은 전례 없이 바로 받아들여지기보다, 공동체의 강력한 지지를 바탕으로 세워집니다. 정책과 지침의 제정 방법으로는 제안을 통한 수립, 기존의 수필 또는 지침의 정책화, 기존의 정책과 지침의 분할 또는 합병을 통한 재구성 등의 여러 방법이 있습니다.\n정책과 지침이 아닌 위키백과 내 운영과 관련된 문서에는 {{수필}}, {{정보문}}, {{위키백과 사용서}} 등을 붙여 구분해야 합니다.\n현재 정책이나 지침으로 제안된 문서는 분류:위키백과 제안에 모여 있습니다. 총의를 통해 채택이 거부된 제안은 분류:위키백과 거부된 제안을 참조하세요. 여러분들의 참여를 환영합니다.' metadata={'source': 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8', 'title': '위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전', 'language': 'ko'}


## Indexing using chromaDB

In [15]:
# Indexing (Texts -> Embedding -> Store)
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings



vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=OpenAIEmbeddings(openai_api_key=API_KEY))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
docs = vectorstore.similarity_search("채택 과정 대해서 설명해주세요.")
print(len(docs))
print(docs[0].page_content)

4
채택 과정
한국어 위키백과에서 오랫동안 확립되어 온 정책과 지침의 대다수는, 영어 위키백과 설립 시 토대가 된 원칙에서 발전된 것들입니다. 물론 타 언어 위키백과의 원칙을 가져오는 것 말고도, 정책과 지침을 일반적인 문제와 문서 훼손 행위의 대응책으로 한국어 위키백과 내 공동체에서 자발적으로 세우기도 했습니다. 정책과 지침 대부분은 전례 없이 바로 받아들여지기보다, 공동체의 강력한 지지를 바탕으로 세워집니다. 정책과 지침의 제정 방법으로는 제안을 통한 수립, 기존의 수필 또는 지침의 정책화, 기존의 정책과 지침의 분할 또는 합병을 통한 재구성 등의 여러 방법이 있습니다.
정책과 지침이 아닌 위키백과 내 운영과 관련된 문서에는 {{수필}}, {{정보문}}, {{위키백과 사용서}} 등을 붙여 구분해야 합니다.
현재 정책이나 지침으로 제안된 문서는 분류:위키백과 제안에 모여 있습니다. 총의를 통해 채택이 거부된 제안은 분류:위키백과 거부된 제안을 참조하세요. 여러분들의 참여를 환영합니다.


In [19]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# LangChain을 구성하기 위한 필수 모듈들 import
# - ChatOpenAI: OpenAI의 GPT 모델과 상호작용
# - ChatPromptTemplate: 프롬프트 템플릿 정의
# - RunnablePassthrough: 입력을 그대로 전달하는 도구
# - StrOutputParser: LLM 출력 문자열을 파싱하는 도구

In [20]:
# Prompt
template = '''Please provide most correct answer from the following context. 
If the answer is not present in the context, please write "The information is not present in the context."
---
Question: {question}
---
Context: {context}
---
Output format:
gradient: ...
Badges: Gluten o
'''

prompt = ChatPromptTemplate.from_template(template)
prompt
# 질문과 컨텍스트를 기반으로 한 LLM의 입력 프롬프트를 정의
# {question}와 {context}는 이후에 실제 데이터로 치환

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Please provide most correct answer from the following context. \nIf the answer is not present in the context, please write "The information is not present in the context."\n---\nQuestion: {question}\n---\nContext: {context}\n'))])

In [ ]:
temp.split('Badesg')

In [21]:
# LLM
model = ChatOpenAI(model='gpt-4o-mini', temperature=0, openai_api_key=API_KEY)
# OpenAI의 GPT-4o-mini 모델을 사용할 LLM 객체 생성
# - 모델 이름: 'gpt-4o-mini|'
# - temperature=0: 출력의 일관성을 높이기 위해 탐색성을 최소화
# - openai_api_key: OpenAI API 인증 키 (미리 정의된 값 필요)

In [22]:
retriever = vectorstore.as_retriever()
# 벡터 스토어(vectorstore)를 기반으로 유사한 문서를 검색하는 retriever 생성
# vectorstore는 위에서 문서 임베딩 데이터를 저장한 chromaDB

In [23]:
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)
# 검색된 문서들(docs)을 하나의 문자열로 결합하는 함수
# - 각 문서의 내용을 개행(\n\n)으로 구분하여 연결

In [24]:
# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    # 컨텍스트(context): retriever에서 검색된 문서를 format_docs로 처리
    # 질문(question): 그대로 전달
    | prompt
    # 질문과 컨텍스트를 프롬프트 템플릿(prompt)에 연결
    | model
    # 프롬프트를 LLM 모델에 전달하여 결과 생성
    | StrOutputParser()
    # LLM의 출력을 문자열로 파싱하여 반환
)


rag_chain.invoke("피해야하는 과정에 대해서 설명해주세요.")
# RAG 체인을 실행하여 질문을 처리
# - retriever가 질문에 대한 유사 문서를 검색
# - 검색된 문서를 프롬프트와 결합하여 모델에 전달
# - 생성된 답변을 문자열로 반환

'피해야 하는 과정에 대해서는 다음과 같은 내용이 있습니다: 소수만 알아듣거나 준법률적인 단어, 지나치게 단순한 표현은 피해야 하며, 지나치게 상투적인 표현이나 일반론도 피해야 합니다. 또한, 과도한 링크는 피하고, 서로 모순되는 내용이 있어서는 안 되며, 정책과 지침은 백과사전의 일부가 아니므로 일반적인 문서와 같은 내용 정책이나 지침을 적용할 필요가 없습니다.'

# PDF RAG

In [25]:
from langchain_community.document_loaders import PyPDFLoader
# PyPDFLoader를 import: PDF 파일을 로드하고 내용을 추출하는 데 사용되는 LangChain의 커뮤니티 문서 로더
# PyPDFLoader는 PDF에서 텍스트를 효율적으로 추출하는 기능을 제공


loader = PyPDFLoader("pdfs/solar_sample.pdf")
# 'solar_sample.pdf' 파일 경로를 사용하여 PyPDFLoader 객체 생성
# pdfs/ 폴더에 있는 solar_sample.pdf 파일을 로드할 준비
# loader 객체를 통해 PDF 내용을 텍스트로 변환하고 문서(Document) 객체로 반환 가능


docs = loader.load()
# PDF 파일을 로드하여 문서(Document) 객체 리스트를 반환
# 각 Document 객체는 PDF의 한 페이지를 나타내며, 페이지 내용(page_content)과 메타데이터(metadata)를 포함
# - loader.load(): PDF 파일의 모든 페이지를 한 번에 로드
# - loader.lazy_load(): 페이지를 필요한 시점에 로드하는 지연 로딩 방식 (메모리 효율성)

print(docs[0].page_content[:1000])
# 첫 번째 페이지의 내용을 출력
# page_content는 문서(Document) 객체의 텍스트 콘텐츠
# [:1000]: 최대 1000자만 출력하여 내용의 일부를 확인

Ignoring wrong pointing object 8 0 (offset 0)


Model Size Type H6 (Avg.) ARC HellaSwag MMLU TruthfulQA Winogrande GSM8K
SOLAR 10.7B-Instruct⇠11B Alignment-tuned74.20 71.08 88.16 66.21 71.43 83.58 64.75Qwen 72B ⇠72B Pretrained 73.60 65.19 85.94 77.37 60.19 82.48 70.43Mixtral 8x7B-Instruct-v0.1⇠47B Instruction-tuned 72.62 70.22 87.63 71.16 64.58 81.37 60.73Yi 34B-200K ⇠34B Pretrained 70.81 65.36 85.58 76.06 53.64 82.56 61.64Yi 34B ⇠34B Pretrained 69.42 64.59 85.69 76.35 56.23 83.03 50.64Mixtral 8x7B-v0.1⇠47B Pretrained 68.42 66.04 86.49 71.82 46.78 81.93 57.47Llama 2 70B ⇠70B Pretrained 67.87 67.32 87.33 69.83 44.92 83.74 54.06Falcon 180B ⇠180B Pretrained 67.85 69.4588.86 70.50 45.47 86.90 45.94SOLAR 10.7B ⇠11B Pretrained 66.04 61.95 84.60 65.48 45.04 83.66 55.50Qwen 14B ⇠14B Pretrained 65.86 58.28 83.99 67.70 49.43 76.80 58.98Mistral 7B-Instruct-v0.2⇠7B Instruction-tuned 65.71 63.14 84.88 60.78 68.26 77.19 40.03Yi 34B-Chat ⇠34B Instruction-tuned 65.32 65.44 84.16 74.90 55.37 80.11 31.92Mistral 7B ⇠7B Pretrained 60.97 59.98 83.31 64.

In [26]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model='gpt-4o-mini', temperature=0, openai_api_key=API_KEY)

prompt_template = PromptTemplate.from_template(
    """
    Please provide most correct answer from the following context. 
    If the answer is not present in the context, please write "The information is not present in the context."
    ---
    Question: {question}
    ---
    Context: {Context}
    """
)
# PromptTemplate 객체 생성
# - 입력 템플릿 정의: 질문과 컨텍스트를 결합하여 프롬프트를 구성
# - {question}: 사용자가 입력할 질문
# - {Context}: 검색된 문서나 추가 정보가 삽입될 자리
# - 이 템플릿은 RAG (Retrieval-Augmented Generation)에서 사용됨

chain = prompt_template | model | StrOutputParser()
# 체인(Chain) 구성: LangChain의 체인 연산자를 사용하여 여러 컴포넌트를 연결
# 1. prompt_template: 프롬프트 생성 (질문과 컨텍스트 삽입)
# 2. model: LLM에 프롬프트 전달 및 답변 생성
# 3. StrOutputParser: 모델의 출력을 문자열로 변환하여 반환

chain

PromptTemplate(input_variables=['Context', 'question'], template='\n    Please provide most correct answer from the following context. \n    If the answer is not present in the context, please write "The information is not present in the context."\n    ---\n    Question: {question}\n    ---\n    Context: {Context}\n    ')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x353b262c0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x353b44070>, model_name='gpt-4o-mini', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()

## Test

In [27]:
chain.invoke({"question": "Explain Table 2?", "Context": docs})

'Table 2 presents evaluation results from the Open LLM Leaderboard for the SOLAR 10.7B and SOLAR 10.7B-Instruct models, along with other top-performing models. It includes scores for six tasks and the H6 score, which is the average of these tasks. The table also indicates the size of the models in billions of parameters and categorizes them based on their training stage as either Pretrained, Instruction-tuned, or Alignment-tuned. Models based on SOLAR 10.7B are highlighted in purple, and the best scores for both H6 and individual tasks are shown in bold.'

In [28]:
chain.invoke({"question": "What is MMLU scores of SOLAR 10.7B?", "Context": docs})

'The MMLU score of SOLAR 10.7B is 84.60.'

In [29]:
chain.invoke({"question": "What is ARC of Mistral?", "Context": docs})

'The information is not present in the context.'

## Simple chatbot

In [30]:
def chatbot(docs):
    print("챗봇을 시작합니다. 'exit'를 입력하면 종료됩니다.")
    while True:
        question = input("질문: ")
        if question.lower() == 'exit':
            print("챗봇을 종료합니다.")
            break

        context = docs
        response = chain.invoke({"question": question, "Context": context})
        print("답변:", response)

# 챗봇 실행
chatbot(docs)


챗봇을 시작합니다. 'exit'를 입력하면 종료됩니다.
질문: What is MMLU scores of Falcon 180B?
답변: The MMLU score of Falcon 180B is 88.86.
질문: What is ARC of Mistral?
답변: The information is not present in the context.
질문: exit
챗봇을 종료합니다.


# Smart RAG

In [31]:
solar_summary = """
SOLAR 10.7B: Scaling Large Language Models with Simple yet Effective Depth Up-Scaling

We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
demonstrating superior performance in various natural language processing (NLP) tasks. 
Inspired by recent efforts to efficiently up-scale LLMs, 
we present a method for scaling LLMs called depth up-scaling (DUS), 
which encompasses depthwise scaling and continued pretraining.
In contrast to other LLM up-scaling methods that use mixture-of-experts, 
DUS does not require complex changes to train and inference efficiently. 
We show experimentally that DUS is simple yet effective 
in scaling up high-performance LLMs from small ones. 
Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
a variant fine-tuned for instruction-following capabilities, 
surpassing Mixtral-8x7B-Instruct. 
SOLAR 10.7B is publicly available under the Apache 2.0 license, 
promoting broad access and application in the LLM field.
"""

In [32]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model='gpt-4o-mini', temperature=0, openai_api_key=API_KEY)

prompt_template = PromptTemplate.from_template(
    """
    Please provide answer from the following context. 
    If the answer is not present in the context, please write "The information is not present in the context."

    ---
    Question: {question}
    ---
    Context: {context}
    """
)
chain = prompt_template | model | StrOutputParser()

chain.invoke({"question": "What is DUS?", "context": solar_summary})

'DUS stands for depth up-scaling, which is a method for scaling large language models (LLMs) that includes depthwise scaling and continued pretraining.'

In [33]:
chain.invoke({"question": "How to get to Seoul from SF", "context": solar_summary})

'The information is not present in the context.'

In [34]:
# RAG or Search?
def is_in(question, context):
    is_in_conetxt = """As a helpful assistant, 
please use your best judgment to determine if the answer to the question is within the given context. 
If the answer is present in the context, please respond with "yes". 
If not, please respond with "no". 
Only provide "yes" or "no" and avoid including any additional information. 
Please do your best. Here is the question and the context:
---
CONTEXT: {context}
---
QUESTION: {question}
---
OUTPUT (yes or no):"""

    is_in_prompt = PromptTemplate.from_template(is_in_conetxt)
    chain = is_in_prompt | model | StrOutputParser()

    response = chain.invoke({"context": context, "question": question})
    print(response)
    return response.lower().startswith("yes")

In [35]:
is_in("How to get to Seoul from SF", solar_summary)

no


False

In [36]:
import os
from tavily import TavilyClient


def smart_rag(question, context):
    # 함수 정의: `smart_rag`는 질문과 초기 컨텍스트를 받아 RAG 기반의 답변을 반환.
    # 매개변수:
    # - `question`: 사용자가 입력한 질문.
    # - `context`: 질문에 대한 기존의 컨텍스트(문서, 정보 등).

    if not is_in(question, context):
        # 조건: 질문에 대한 답변이 기존 `context`에 포함되어 있지 않은 경우.
        # `is_in`: 사용자가 정의한 함수로 보이며, 질문에 대한 답변이 컨텍스트에 있는지 확인.

        print("Searching in tavily")
        tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
        # Tavily API 클라이언트를 생성.
        # 환경 변수에서 API 키를 가져와 `TavilyClient`에 전달.
        # - `os.environ["TAVILY_API_KEY"]`: 환경 변수에 저장된 Tavily API 키.

        context = tavily.search(query=question)
        # Tavily 검색 엔진을 사용하여 질문(`question`)과 관련된 컨텍스트를 검색.
        # `context` 변수에 검색된 문서나 정보를 저장.


    chain = prompt_template | model | StrOutputParser()
    # LangChain 체인을 생성:
    # - `prompt_template`: 질문과 컨텍스트를 조합한 프롬프트 생성.
    # - `model`: GPT 모델로 질문과 컨텍스트를 전달하여 답변 생성.
    # - `StrOutputParser()`: 모델 출력 결과를 문자열로 변환.

    return chain.invoke({"context": context, "question": question})
    # 체인을 실행하여 최종 답변을 반환:
    # - `context`: 검색된 컨텍스트.
    # - `question`: 사용자 질문.


In [37]:
smart_rag("How to get to Seoul from SF?", solar_summary)

no
Searching in tavily


'To get to Seoul from San Francisco, you can fly from San Francisco International Airport (SFO) to Incheon International Airport (ICN). There are various options available, including comparing ticket prices and travel times through travel planners like Rome2Rio. For more details, you can check the following resources:\n\n1. [Rome2rio](https://www.rome2rio.com/s/San-Francisco/Seoul)\n2. [Google Flights](https://www.google.com/travel/flights/flights-from-san-francisco-to-seoul.html)\n3. [Skyscanner](https://www.skyscanner.com/routes/sfo/sela/san-francisco-international-to-seoul.html)\n4. [KAYAK](https://www.kayak.com/flight-routes/San-Francisco-SFO/Seoul-SEL)\n5. [Expedia](https://www.expedia.com/lp/flights/sfo/icn/san-francisco-to-seoul)\n\nThese resources can help you find flights and plan your trip effectively.'

In [56]:
smart_rag("What ingredient of rose tuckboki?", solar_summary)

no
Searching in tavily


'The main ingredients of rose tteokbokki are gochujang, cream, soy sauce, and tteokbokki-tteok (rice cakes).'

# ReACT

In [41]:
from langchain_community.tools.tavily_search.tool import TavilySearchResults
# LangChain의 커뮤니티 도구에서 Tavily 검색 도구(TavilySearchResults)를 가져옴.
# TavilySearchResults는 질문에 대해 Tavily 검색 엔진을 호출하고 관련 문서를 반환하는 도구.


tools = [TavilySearchResults(max_results=5)]
# TavilySearchResults 도구를 사용하여 검색 기능을 구성:
# - max_results=5: 검색 결과의 최대 개수를 5개로 제한.
# - TavilySearchResults 객체를 리스트로 저장. LangChain 워크플로에서 사용할 수 있도록 준비.
tools

[TavilySearchResults()]

In [42]:
from langchain import hub
from langchain.tools.render import render_text_description
# `hub`: LangChain의 허브를 사용하여 사전에 정의된 리소스(모델, 프롬프트 등)를 가져오는 모듈.
# `render_text_description`: LangChain의 도구에 대한 텍스트 설명을 생성하는 유틸리티 함수.
# - 도구들의 기능 및 목적을 간결하게 서술하는 데 사용.

prompt = hub.pull("hwchase17/react-chat")
# LangChain 허브에서 "hwchase17/react-chat" 프롬프트를 가져옴.
# - "hwchase17/react-chat": ReAct(Reinforcement Learning with Action and Thought) 방식으로 설계된 사전 정의된 프롬프트.
# - `hub.pull()`은 LangChain 허브에서 특정 리소스를 불러오는 메서드.
# 이 프롬프트는 주로 도구와 상호작용하는 챗봇을 위한 템플릿을 포함.

prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)
# 가져온 프롬프트를 `partial()` 메서드를 사용하여 커스터마이징.
# - `tools`: LangChain 도구에 대한 설명을 텍스트로 생성한 결과.
# - `tool_names`: 도구들의 이름을 쉼표로 구분한 문자열 형태로 생성.
# 결과적으로, 이 프롬프트는 특정 도구 정보(`tools`와 `tool_names`)를 포함하여 재정의됨.

/Users/notaehyeong/miniforge3/envs/llm/lib/python3.10/site-packages/langchain/hub.py:83: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [43]:
from langchain.memory import ConversationBufferMemory
# LangChain의 `ConversationBufferMemory` 클래스를 import.
# `ConversationBufferMemory`는 대화 내용을 메모리(버퍼)에 저장하여, 이전 대화 기록을 유지하고 사용할 수 있도록 지원.


memory = ConversationBufferMemory(memory_key="chat_history")
# `ConversationBufferMemory` 객체를 생성:
# - `memory_key`: 저장된 대화 기록을 참조할 때 사용할 키. 여기서는 "chat_history"로 설정.
# - 이 메모리는 LangChain 워크플로에서 사용되며, 사용자와의 대화 기록을 관리.

In [44]:
model = ChatOpenAI(model='gpt-4o-mini', temperature=0, openai_api_key=API_KEY)
model_with_stop = model.bind(stop=["\nObservation"])
# `model.bind()` 메서드는 기존 모델(`model`)의 일부 매개변수(파라미터)를 고정한 새로운 모델 객체를 생성.
# - `stop`: 모델이 응답을 생성할 때 멈추는 조건(stop tokens)을 지정.
# - `["\nObservation"]`: 모델이 텍스트를 생성하는 동안 `"\nObservation"`이 등장하면 응답 생성을 중단.

In [45]:
from langchain.agents import AgentExecutor
# AgentExecutor는 LangChain에서 에이전트를 실행하는 데 사용되는 클래스.
# - 에이전트: LLM과 도구를 결합하여 지능적인 작업을 수행하는 구성 요소.
# AgentExecutor는 이러한 에이전트와 연관된 도구 및 메모리를 사용해 체계를 실행.

from langchain.agents.format_scratchpad import format_log_to_str
# `format_log_to_str`는 중간 단계의 로그(intermediate_steps)를 문자열로 변환하는 유틸리티 함수.
# - ReAct 방식에서 모델이 생성한 중간 사고 과정(Thought)과 관찰(Observation)을 정리.

from langchain.agents.output_parsers import ReActSingleInputOutputParser
# `ReActSingleInputOutputParser`는 ReAct 방식에서 단일 입력/출력을 처리하는 파서.
# - 입력: 질문 및 컨텍스트.
# - 출력: 모델의 최종 응답.
# ReAct는 모델이 "사고(Thought)"와 "행동(Action)" 단계를 거쳐 결과를 생성하는 구조를 사용.


agent = (
    {
        "input": lambda x: x["input"],
        # 사용자 입력에서 "input" 키를 추출하여 전달.
        
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        # 중간 단계(`intermediate_steps`)를 문자열로 변환하여 `agent_scratchpad`에 저장.
        
        "chat_history": lambda x: x["chat_history"],
        # 이전 대화 기록(`chat_history`)를 유지하여 대화형 작업에서 컨텍스트로 사용.
    }
    | prompt
    # 사용자 입력(`input`), 중간 단계(`agent_scratchpad`), 대화 기록(`chat_history`)을 프롬프트와 결합.

    | model_with_stop
    # 프롬프트를 GPT 모델(`model_with_stop`)에 전달하여 응답 생성.

    | ReActSingleInputOutputParser()
    # 생성된 응답을 ReAct 스타일의 출력으로 파싱.
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
)
# AgentExecutor는 위에서 정의한 에이전트와 관련 도구, 메모리를 결합하여 실행 가능한 객체 생성.
# 매개변수:
# - `agent`: 위에서 정의한 에이전트. 입력 처리, 프롬프트 생성, 모델 호출, 출력 파싱을 포함.
# - `tools`: 에이전트가 사용할 LangChain 도구 리스트. (예: 검색 도구, 계산 도구 등.)
# - `memory`: 대화 메모리 객체. (이전 대화 내용을 참조하여 응답의 일관성을 유지.)
# - `verbose=True`: 실행 중 상세 로그를 출력하여 디버깅 및 상태 확인 가능.


In [46]:
agent_executor.invoke({"input": '세종대왕이 누구야'})



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? No
Final Answer: 세종대왕(세종대왕, 1397-1450)은 조선의 제4대 왕으로, 한글을 창제한 것으로 가장 잘 알려져 있습니다. 그는 1418년부터 1450년까지 통치하였으며, 그의 통치 기간 동안 과학, 기술, 문화, 농업 등 여러 분야에서 많은 발전이 있었습니다. 세종대왕은 백성을 사랑하고 그들의 삶을 개선하기 위해 노력한 왕으로 평가받고 있습니다. 그의 업적 중에는 천문학, 의학, 음악 등 다양한 분야에서의 기여가 포함되어 있습니다.
```

> Finished chain.


{'input': '세종대왕이 누구야',
 'chat_history': '',
 'output': '세종대왕(세종대왕, 1397-1450)은 조선의 제4대 왕으로, 한글을 창제한 것으로 가장 잘 알려져 있습니다. 그는 1418년부터 1450년까지 통치하였으며, 그의 통치 기간 동안 과학, 기술, 문화, 농업 등 여러 분야에서 많은 발전이 있었습니다. 세종대왕은 백성을 사랑하고 그들의 삶을 개선하기 위해 노력한 왕으로 평가받고 있습니다. 그의 업적 중에는 천문학, 의학, 음악 등 다양한 분야에서의 기여가 포함되어 있습니다.\n```'}

In [52]:
agent_executor.invoke({"input": '한양대학교 데이터사이언스학과 정보 알려줘?'})



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? No
Final Answer: 한양대학교 데이터사이언스학과는 데이터 분석, 머신러닝, 인공지능, 빅데이터 처리 등 다양한 분야를 연구하고 교육하는 학부입니다. 이 학과는 데이터의 수집, 처리, 분석 및 활용에 대한 이론과 실습을 제공하며, 학생들이 데이터 기반의 문제 해결 능력을 기를 수 있도록 돕습니다. 또한, 산업계와의 협력을 통해 실무 경험을 쌓을 수 있는 기회를 제공하고 있습니다. 더 구체적인 정보는 한양대학교 공식 웹사이트나 데이터사이언스학과의 페이지를 통해 확인할 수 있습니다.
```

> Finished chain.


{'input': '한양대학교 데이터사이언스학과 정보 알려줘?',
 'chat_history': 'Human: 세종대왕이 누구야\nAI: 세종대왕(세종대왕, 1397-1450)은 조선의 제4대 왕으로, 한글을 창제한 것으로 가장 잘 알려져 있습니다. 그는 1418년부터 1450년까지 통치하였으며, 그의 통치 기간 동안 과학, 기술, 문화, 농업 등 여러 분야에서 많은 발전이 있었습니다. 세종대왕은 백성을 사랑하고 그들의 삶을 개선하기 위해 노력한 왕으로 평가받고 있습니다. 그의 업적 중에는 천문학, 의학, 음악 등 다양한 분야에서의 기여가 포함되어 있습니다.\n```\nHuman: 한양대학교 한경삭 교수님이 누구야??\nAI: 한양대학교의 한경삭 교수님은 데이터사이언스학부에서 인간 중심 인공지능 및 데이터사이언스 분야를 전문으로 연구하고 가르치고 계십니다. 교수님의 연구는 인간-컴퓨터 상호작용과 관련된 주제들을 포함하고 있습니다. 추가적인 정보는 한양대학교의 공식 웹사이트에서 확인할 수 있습니다.\nHuman: 한양대학교 한경식 교수님이 누구야??\nAI: 한양대학교의 한경식 교수님은 데이터사이언스학부의 부교수이자 학부장으로, 인간-컴퓨터 상호작용 및 인간 중심 인공지능, 데이터사이언스 분야를 전문으로 연구하고 가르치고 계십니다. 교수님의 연구는 컴퓨터 기술과 인간 활동 간의 관계를 탐구하며, 사람에게 도움이 되는 기술 개발에 중점을 두고 있습니다. 연락처는 02-2220-2517이며, 이메일은 kyungsikhan@hanyang.ac.kr입니다.',
 'output': '한양대학교 데이터사이언스학과는 데이터 분석, 머신러닝, 인공지능, 빅데이터 처리 등 다양한 분야를 연구하고 교육하는 학부입니다. 이 학과는 데이터의 수집, 처리, 분석 및 활용에 대한 이론과 실습을 제공하며, 학생들이 데이터 기반의 문제 해결 능력을 기를 수 있도록 돕습니다. 또한, 산업계와의 협력을 통해 실무 경험을 쌓을 수 있는 기회를 제공하고 있습니다. 더 구체적인 정보는 한양대학교 공